In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
import copy
import itertools

In [2]:
base_price=[[10,10,10],
            [10,10,10]]

intercept = [23,20]
coef_1 =[-6,-5]
coef_2 =[0.01,0.02]
coef_3 = [1,1.2]
coef_4 = [-0.003,-0.004]

target_trade_spend = [276860,223573]
target_edlp_spend = [63366,35525]
target_tpr_spend = [213494,188048]

# target_trade_spend = 10515.58
# intercept = 15
# coef_1 =-3
# coef_2 =0.004
# coef_3 =1
# coef_4 = -0.003
# target_edlp_spend = 1906.41
# target_tpr_spend = 8609.172
# init = [1,1,0]
idx = range(6)
products = 2
weeks = 3
tot = products*weeks

In [25]:
def cal_sales_price(edlp, tpr, flag, j):
    unit_sales=[]
    for i in range(weeks*j,weeks*(j+1)):
#         print(i,edlp[i], edlp[(i+3)%6], edlp)
        a = (log( edlp[i])*coef_1[j])* flag[i]
        b = (log(base_price[j][i%weeks])*coef_1[j]+ tpr[i]*coef_2[j])*(1- flag[i])
        
        c = (log(edlp[(i+3)%6])*coef_3[j])* flag[(i+3)%6]
        d = (log(base_price[(j+1)%2][i%weeks])*coef_3[j]+ tpr[(i+3)%6]*coef_4[j])*(1- flag[(i+3)%6])
        
        unit_sales.append(exp( a+ b+c+d+intercept[j]))
                
    price = []
    for i in range(weeks*j,weeks*(j+1)):
        
        price.append((base_price[j][i%weeks]*(1-tpr[i]/100)*(1-flag[i]))  +  (edlp[i]*flag[i]))
    
    return unit_sales, price
def calc_dollar_sales(edlp, tpr, flag):
    dollar_sales=[]
#     lst = [edlp, tpr]
#     print(lst[0])
    for j in range(products):
        unit_sales, price = cal_sales_price(edlp, tpr, flag, j)

        dollar_sales_single = [unit_sales[i]*price[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
    
    return dollar_sales

def calc_total_trade_spent(edlp, tpr, flag):
    total_trade_spent = []
    for j in range(products):
        unit_sales, price = cal_sales_price(edlp, tpr, flag,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp, tpr, flag):
    edlp_trade_spent = []
    
    for j in range(products):
        unit_sales, price = cal_sales_price(edlp, tpr, flag,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]

        edlp_trade_spent_partial = [total_trade_spent_partial[i]*flag[i+j*weeks] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent

def calc_flag_util(flag):
    flag_util = [flag[i]*(1-flag[i]) for i in idx]
    return flag_util

In [26]:
def create_model(init):
    print("Init value= ",init)
    def initial(model,i):
        return init[i]

    model = ConcreteModel(name='Spend_Optim')
    model.edlp = Var(idx, initialize=10, bounds=(9.5, 10), domain=NonNegativeReals)
    model.tpr = Var(idx, initialize=5, bounds=(5, 50), domain=NonNegativeReals)
    model.flag = Var(idx,initialize=initial,domain=Binary)
    
    dollar_sales = calc_dollar_sales(model.edlp, model.tpr, model.flag)
    total_trade_spent = calc_total_trade_spent(model.edlp, model.tpr, model.flag)
    edlp_trade_spent = calc_edlp_trade_spent(model.edlp, model.tpr, model.flag)
    
    model.obj = Objective(expr=sum(dollar_sales[0])+sum(dollar_sales[1]) ,sense= maximize)
    model.c1 = Constraint(expr = sum(total_trade_spent[0]) == target_trade_spend[0])
    model.c2 = Constraint(expr = sum(total_trade_spent[1]) == target_trade_spend[1])
    model.c3 = Constraint(expr = sum(edlp_trade_spent[0]) == target_edlp_spend[0] )
    model.c4 = Constraint(expr = sum(edlp_trade_spent[1]) == target_edlp_spend[1] )
#     model.c5 = Constraint(expr = sum(calc_flag_util(model.flag))==0 )
    return model

In [27]:
def call_solver(init, name='bonmin'):
    assert type(init) == list
    model = create_model(init)
    opt_1 = SolverFactory(name)
    results_1 = opt_1.solve(model)
    model.display()
#     print(results_1)
#     print(get_info_from_results(results_1, 'Time: '))

combinations=[[0,0,0,0,1,1]]    
for c in combinations:
    call_solver(c, 'bonmin')
    break

Init value=  [0, 0, 0, 0, 1, 1]
edlp
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.50563219866149 :    10 : False : False : NonNegativeReals
          1 :   9.5 : 9.750000002499998 :    10 : False : False : NonNegativeReals
          2 :   9.5 : 9.750000002499998 :    10 : False : False : NonNegativeReals
          3 :   9.5 :      9.7500000025 :    10 : False : False : NonNegativeReals
          4 :   9.5 :      9.7500000025 :    10 : False : False : NonNegativeReals
          5 :   9.5 : 9.607149465629806 :    10 : False : False : NonNegativeReals
    tpr : Size=6, Index=tpr_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     5 :       27.500000225 :    50 : False : False : NonNegativeReals
          1 :     5 : 10.929218938931964 :    50 : False : False : NonNegativeReals
          2 :     5 :  9.561578253215089 :    50

In [28]:
def cal_sales_price(edlp, tpr, flag, j):
    unit_sales=[]
    for i in range(weeks*j,weeks*(j+1)):
#         print(i,edlp[i], edlp[(i+3)%6], edlp)
        a = (np.log( edlp[i])*coef_1[j])* flag[i]
        b = (np.log(base_price[j][i%weeks])*coef_1[j]+ tpr[i]*coef_2[j])*(1- flag[i])
        
        c = (np.log(edlp[(i+3)%6])*coef_3[j])* flag[(i+3)%6]
        d = (np.log(base_price[(j+1)%2][i%weeks])*coef_3[j]+ tpr[(i+3)%6]*coef_4[j])*(1- flag[(i+3)%6])
        
        unit_sales.append(np.exp( a+ b+c+d+intercept[j]))
                
    price = []
    for i in range(weeks*j,weeks*(j+1)):
        
        price.append((base_price[j][i%weeks]*(1-tpr[i]/100)*(1-flag[i]))  +  (edlp[i]*flag[i]))
    
    return unit_sales, price


def calc_dollar_sales(edlp, tpr, flag):
    dollar_sales=[]
    for j in range(products):
        unit_sales, price = cal_sales_price(edlp, tpr, flag, j)

        dollar_sales_single = [unit_sales[i]*price[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
    
    return dollar_sales



def calc_total_trade_spent(edlp, tpr, flag):
    total_trade_spent = []
    for j in range(products):
        unit_sales, price = cal_sales_price(edlp, tpr, flag,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp, tpr, flag):
    edlp_trade_spent = []
    
    for j in range(products):
        unit_sales, price = cal_sales_price(edlp, tpr, flag,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]

        edlp_trade_spent_partial = [total_trade_spent_partial[i]*flag[i+j*weeks] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent

def calc_flag_util(flag):
    flag_util = [flag[i]*(1-flag[i]) for i in idx]
    return flag_util

In [39]:
intercept = [23,20]
coef_1 =[-6,-5]
coef_2 =[0.01,0.02]
coef_3 = [1,1.2]
coef_4 = [-0.003,-0.004]

base_price=[[10,10,10],
            [10,10,10]]

edlp=[9.5,10,10,10,10,9.6]
tpr=[0,5,15,15,5,0]
flag=[1,0,0,0,0,1]

calc_edlp_trade_spent(edlp,tpr,flag)

[[63366.278926377876, 0.0, 0.0], [0.0, 0.0, 35525.139769282396]]

In [31]:


# print(calc_dollar_sales(edlp,tpr,flag), end=' ')
# print(calc_total_trade_spent(edlp,tpr,flag),end=' ')
# calc_edlp_trade_spent(edlp,tpr,flag)
z = pd.DataFrame()
z['product'] = ['A']*3 + ['B']*3
z['base_price'] = list(itertools.chain.from_iterable(base_price))
z['edlp'] = edlp
z['tpr'] = tpr
z['flag'] = flag
z['c_edlp'] = np.roll(z['edlp'],3)
z['c_tpr'] = np.roll(z['tpr'],3)
z['unit_sales'] = cal_sales_price(edlp,tpr,flag,0)[0] + cal_sales_price(edlp,tpr,flag,1)[0]
z['price'] = cal_sales_price(edlp,tpr,flag,0)[1] + cal_sales_price(edlp,tpr,flag,1)[1]
z['dollar_sales'] = list(itertools.chain.from_iterable(calc_dollar_sales(edlp,tpr,flag)))
z['trade_spent'] = list(itertools.chain.from_iterable(calc_total_trade_spent(edlp,tpr,flag)))
z['edlp_spent'] = list(itertools.chain.from_iterable(calc_edlp_trade_spent(edlp,tpr,flag)))
z

,product,base_price,edlp,tpr,flag,c_edlp,c_tpr,unit_sales,price,dollar_sales,trade_spent,edlp_spent
0,A,10,9.5,0,1,10.0,15,126732.557853,9.5,1.203959e+06,63366.278926,63366.278926
1,A,10,10.0,5,0,10.0,5,100919.105073,9.5,9.587315e+05,50459.552537,0.000000
2,A,10,10.0,15,0,9.6,0,108689.724792,8.5,9.238627e+05,163034.587188,0.000000
3,B,10,10.0,15,0,9.5,0,97599.211489,8.5,8.295933e+05,146398.817233,0.000000
4,B,10,10.0,5,0,10.0,5,83297.735809,9.5,7.913285e+05,41648.867905,0.000000
5,B,10,9.6,0,1,10.0,15,88812.849423,9.6,8.526034e+05,35525.139769,35525.139769


In [18]:
calc_dollar_sales(edlp,tpr,flag)

[[1203959.2996011795, 958731.498194816, 923862.6607326979],
 [829593.297652488, 791328.4901867423, 852603.3544627768]]